In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import pickle
import pandas as pd

import statsmodels.stats.api as sms

In [30]:
with open("mldb_2021-07-16.pickle", 'rb') as f:
    evals = pickle.load(f)

In [31]:
df = pd.DataFrame(evals)

In [32]:
# Ignore checkpoints for now
df = df[df.epoch == -1]

In [33]:
# Add metrics
df['accuracy'] = df.stats.apply(lambda x: x['num_correct_and_size'][0] / x['num_correct_and_size'][1])

num_correct_vals = df.stats.apply(lambda x: x['num_correct_and_size'][0]).values
size_vals = df.stats.apply(lambda x: x['num_correct_and_size'][1]).values
cis = sms.proportion_confint(num_correct_vals, size_vals, alpha=0.05, method='beta')
df['accuracy_ci'] = list(zip(*cis))

In [34]:
# Pairs of id-train, id-test
ID_PAIRS = [
    ("cifar10-train", "cifar10-test"),
    ("cifar10-train", "cifar10-STL10classes"),
    ("FMoW-train", "FMoW-id_test"),
    ("FMoW-train", "FMoW-id_val"),
    # TODO: Add more here
]

In [35]:
def reformat(_df, train, test):
    test_eval = _df[_df.test_set == test]
    if len(test_eval) == 0:
        return pd.DataFrame()
    test_eval = test_eval.iloc[0]
    shift_evals = _df[~_df.test_set.isin([train, test])]
    newdf = shift_evals[["model_family", "model_id", "epoch", "rule_params"]]
    newdf = newdf.rename(columns={"rule_params": "hyperparameters"})
    newdf["train_set"] = train
    newdf["test_set"] = test
    newdf["shift_set"] = shift_evals["test_set"]
    newdf["test_accuracy"] = test_eval.accuracy
    newdf["test_accuracy_ci"] = [test_eval.accuracy_ci for _ in range(len(newdf))]
    newdf["shift_accuracy"] = shift_evals["accuracy"]
    newdf["shift_accuracy_ci"] = shift_evals["accuracy_ci"]
    return newdf

In [36]:
new_df = []
for train, test in ID_PAIRS:
    df_train = df[df.train_set == train]
    shift_sets = set(df_train.test_set) - set([train, test])
    new_df.extend([reformat(modeldf, train, test) for _, modeldf in df_train.groupby("model_id")])

In [37]:
new_df = pd.concat(new_df)

In [38]:
def rename_model_family(model_family):
    if "RandFeatures" in model_family:
        return "RandomFeatures"
    elif "K_nearest_neighbors" in model_family:
        return "KNN"
    return model_family

new_df["model_family"] = new_df.model_family.apply(rename_model_family)

In [39]:
def rename_test_set(ts):
    if ts == "cifar10-STL10classes":
        return "cifar10-test-STL10classes"
    return ts
new_df["test_set"] = new_df.test_set.apply(rename_test_set)

In [40]:
new_df.to_csv("results.csv", index=False)